# Data Cleaning

This notebook presents the whole data cleaning process, which consists in extracting new tables and relations, as well as cleaning the existing files from dirty tuples and values.

The new clean data files are saved in the `.csv` format, and will be used to load data to the database.

In [3]:
# Import packages
import pandas as pd
import os
import numpy as np
import csv
import json
import utils

## Data loading

We first import all the `.csv` files into `pandas` DataFrames.

_Note_: some lines are ill-formed, we choose to ignore them.

In [4]:
# Root of the data files
PATH = os.path.join('..', 'data', 'original')

# Dic: name -> dataframe
dataframes = {}

# Get all the original files
for file in os.listdir(PATH):
    # Skip hidden files
    if (file.startswith('.')):
        continue
        
    name = file.split('.')[0]
    # Note: some lines are ill-formed, we ignore them
    dataframes[name] = pd.read_csv(os.path.join(PATH, file), encoding='utf-8', quoting=csv.QUOTE_NONE)

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We just remove here all stories that have everything null appart from _type-id issue-id_ and _id_.

In [5]:
dataframes['story'] = dataframes['story'].dropna(thresh=4)

## Notes table

This part aims to extract the notes from each table containing a `notes` attributes. Notes are loaded in a new table, and replaced by foreign keys in the original tables.


We first concatenate all the notes from all the dataframes. And create the new note dataframe

In [6]:
notes = pd.Series()

# Get all the notes from all the dataframes containing notes
for _, df in dataframes.items():
    if 'notes' in df.columns: 
        notes = notes.append(df['notes'].dropna(), ignore_index=True)
    
    if 'reprint_notes' in df.columns:
        notes = notes.append(df['reprint_notes'].dropna(), ignore_index=True)

notes_df = utils.extract_table(notes, 'notes')
dataframes['notes'] = notes_df
notes_df.head()


,id,notes
1,1,Lettering credit from Dick Ayers via Mike Quil...
2,2,Lettering credit from Dick Ayers via Mike Quil...
3,3,Single panel cartoons
4,4,Victorian style comic (no word balloons; brief...
5,5,single panel cartoons


Replace the notes by the IDs in the original tables:

In [7]:
for name, df in dataframes.items():
    # Skip the notes dataframe obviously
    if name == 'notes':
        continue
    
    if 'notes' in df.columns:
        # Map notes to their IDs
        df['notes_id'] = utils.map_column(df['notes'], dataframes['notes'], 'id', 'notes')
        df.drop('notes', axis=1, inplace=True)
        
    if 'reprint_notes' in df.columns:
        df['reprint_notes_id'] = utils.map_column(df['reprint_notes'], dataframes['notes'], 'id', 'notes')
        df.drop('reprint_notes', axis=1, inplace=True)

In [8]:
dataframes['story']['notes_id'].head()

0    NaN
1    1.0
2    2.0
3    2.0
4    NaN
Name: notes_id, dtype: float64

In [9]:
dataframes['story']['reprint_notes_id'].head(10)

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5    290970.0
6         NaN
7    290971.0
8    290971.0
9         NaN
Name: reprint_notes_id, dtype: float64

## First/Last issue Relation

We noticed that there is a cyclic dependency between the tables _Issues_ and _Series_, since issues belong to a serie, and series have a first and last issue. It's generally a bad idea (and impossible practically) to create such cyclic relations between tables. So we decide to create a new relation *First_last_issue* to link series with their first and last issue, and remove the reference to _Issues_ in _Series_.

In [10]:
# Extract relation
first_last_issue = dataframes['series'][['id', 'first_issue_id', 'last_issue_id']]

# Rename the columns
first_last_issue.columns = ['serie_id', 'first_issue_id', 'last_issue_id']

# Remove rows if first_issue_id and last_issue_id are both NULL
first_last_issue = first_last_issue.dropna(subset=['first_issue_id', 'last_issue_id'], how='all')

# Save the new relation
dataframes['first_last_issue'] = first_last_issue

first_last_issue.head()

,serie_id,first_issue_id,last_issue_id
0,1,1.0,1.0
1,2,2.0,2.0
2,3,3.0,3.0
3,4,6.0,6.0
4,5,4.0,4.0


We can now drop the *first_issue_id* and *last_issue_id* columns of _Series_

In [11]:
dataframes['series'] = dataframes['series'].drop(['first_issue_id', 'last_issue_id'], axis=1)

## Artists table

We first scan through all the different categories of artists , clean the data and then store all artist in one single table as described in our ER diagram.

In [12]:
# Make table to store the list of all artists
all_artists = pd.Series()
# Dictionnary to store all artists of one category
artists = {}
categories = ['script', 'pencils', 'inks', 'colors', 'letters']

for category in categories:
    # Unpack the artists lists so we have all artists for every story
    unpacked = utils.unpack_column(dataframes['story'], 'id', category)
    
    # Clean the unpacked elements 
    unpacked[category] = utils.clean_column(unpacked[category])
    
    # We have now our relation with story IDs and artists names
    artists[category] = unpacked.dropna(how='any')
    
    # Add artists to the global artist list
    all_artists = all_artists.append(artists[category][category], ignore_index=True)

We can now extract our new artists table from the whole list of artists:

In [13]:
dataframes['artists'] = utils.extract_table(all_artists, 'name')
dataframes['artists'].head()

,id,name
1,1,Gustave Doré
2,2,Harry Rogers
3,3,Wilhelm Busch
4,4,The Donaldson Brothers
5,5,Richard Doyle


Now for each relation of artist, we map the names to the IDs:

In [14]:
for category in categories:
    relation = artists[category]
    relation.columns = ['story_id', 'artist_id']
    relation['artist_id'] = utils.map_column(relation['artist_id'], dataframes['artists'], 'id', 'name')
    dataframes[category] = relation

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
dataframes['script'].head()

,story_id,artist_id
7,13.0,1
8,14.0,1
9,15.0,1
10,16.0,1
11,17.0,1


We can now drop the different artists columns from the original Story dataframe:

In [16]:
dataframes['story'] = dataframes['story'].drop(categories, axis=1)

## Characters table

We are now interested in extracting the characters and building the corresponding relations between _Stories_ and _Characters_. We consider the _characters_ and _feature_ attributes of _Stories_ to be characters, but we build different relationships to keep the original meaning.

Note that some cells contains multiples values, so we need to unpack them, as we did for _Artists_.

In [17]:
all_characters = pd.Series()
char_types = ['feature', 'characters']
char_relations = {}

for c_type in char_types:
    # Extract relation and unpack the lists
    unpacked = utils.unpack_column(dataframes['story'][['id', c_type]], 'id', c_type)
    
    # Clean values
    unpacked[c_type] = utils.clean_column(unpacked[c_type])
    unpacked = unpacked.dropna(how='any')
    
    # We got our clean and unpacked relation for each type
    char_relations[c_type] = unpacked
    
    # Accumulate characters
    all_characters = all_characters.append(unpacked[c_type], ignore_index=True)

In [18]:
char_relations['characters'].head()

,id,characters
39,45.0,John Mishler
1875358,60.0,Maurice
54,60.0,Max
55,61.0,Max
1875359,61.0,Maurice


In [19]:
# Build the Characters table
dataframes['characters'] = utils.extract_table(all_characters, 'name')
dataframes['characters'].head()

,id,name
1,1,Rawhide Kid
2,2,Max and Maurice
3,3,Brown Jones and Robinson
4,4,Plish and Plum
5,5,Daral


We can now replace values by IDs in the relations:

In [20]:
for c_type in char_types:
    relation = char_relations[c_type]
    relation.columns = ['story_id', 'character_id']
    relation['character_id'] = utils.map_column(relation['character_id'], dataframes['characters'], 'id', 'name')
    
    if c_type == 'feature':
        name = 'stories_features'
    elif c_type == 'characters':
        name = 'stories_characters'
        
    dataframes[name] = relation

In [21]:
dataframes['stories_characters'].head()

,story_id,character_id
39,45.0,85178
1875358,60.0,85179
54,60.0,34106
55,61.0,34106
1875359,61.0,85179


In [22]:
dataframes['stories_features'].head()

,story_id,character_id
0,6.0,1
1,7.0,1
2,8.0,1
3,9.0,1
54,60.0,2


We can now delete the features and the character columns from the story dataframe

In [23]:
dataframes['story'] = dataframes['story'].drop(char_types, axis=1)

## Editors table 

We are interested in creating a spearate table for all the editor of both the _Stories_ and the _Issues_. We will however create two relations, one for each table. There can be multiple editors per item we therfore need to unpack the columns

In [24]:
all_editors = pd.Series()
dfs = ['story','issue']
editors_relations = {}

for df in dfs:
    # Extract relation and unpack the lists
    unpacked = utils.unpack_column(dataframes[df][['id', 'editing']], 'id', 'editing')
    
    # Clean values
    unpacked['editing'] = utils.clean_column(unpacked['editing'])
    unpacked = unpacked.dropna(how='any')
    
    # We got our clean and unpacked relation for each type
    editors_relations[df] = unpacked
    
    # Accumulate characters
    all_editors = all_editors.append(unpacked['editing'], ignore_index=True)

In [25]:
editors_relations['story'].head()

,id,editing
393,399.0,Norbert Wallez
469,477.0,Norbert Wallez
475,485.0,Norbert Wallez
713,832.0,Percy Crosby
1063,1197.0,Clifford McBride


In [26]:
# Build the Editors table
dataframes['editors'] = utils.extract_table(all_editors, 'name')
dataframes['editors'].head()

,id,name
1,1,Norbert Wallez
2,2,Percy Crosby
3,3,Clifford McBride
4,4,Gene Aherne
5,5,J.R.Williams


We now map the editors relation table to the editor table

In [27]:
for df in dfs:
    relation = editors_relations[df]
    relation.columns = [df+'_id', 'editor_id']
    relation['editor_id'] = utils.map_column(relation['editor_id'], dataframes['editors'], 'id', 'name')
    
    if df == 'story':
        name = 'stories_editing'
    elif df == 'issue':
        name = 'issues_editing'
        
    dataframes[name] = relation

In [28]:
dataframes['stories_editing'].head()

,story_id,editor_id
393,399.0,1
469,477.0,1
475,485.0,1
713,832.0,2
1063,1197.0,3


In [29]:
dataframes['issues_editing'].head()

,issue_id,editor_id
0,1,2133
2,3,2134
4,5,2134
9,10,2134
10,11,2134


We can now delete the editing column form both issue and story dataframes

In [30]:
dataframes['story'] = dataframes['story'].drop('editing', axis=1)
dataframes['issue'] = dataframes['issue'].drop('editing', axis=1)

## Individual files cleaning

This part aims to clean each `.csv` file individually in order to remove dirty rows and clear values that need some special treatment.

### Country

By browsing the country data, we see that one row is not valid, with ID 248. We see in the cell below that for `publisher`, for example, no row references this ID, which is with high probably pure dirty data, we can safely remove it.

In [31]:
pub = dataframes['publisher']
print('Number of publisher with country_id 248: {}.'.format(len(pub[pub['country_id'] == 248])))

# Look for NaN values
print('NaN values: ')
df = dataframes['country']
df.isnull().sum()

Number of publisher with country_id 248: 0.
NaN values: 


id      0
code    0
name    0
dtype: int64

In [32]:
# Remove the desired row
dataframes['country'] = df[df['id'] != 248]

### Story Reprint

The story reprint table needs to be full, as we don't accept _NULL_ foreign keys in this case. We see in the cell below that there are no empty cells in the table.

In [33]:
dataframes['story_reprint'].isnull().sum()

id           0
origin_id    0
target_id    0
dtype: int64

### Story Type

By looking at the story types we see that the third row is problematic:

In [34]:
df = dataframes['story_type']
df.ix[2]

id                                             3
name    (backcovers) *do not use* / *please fix*
Name: 2, dtype: object

We check if any story contains a reference to this row:

In [35]:
stories = dataframes['story']
print('Number of stories referencing ID 3: {}.'.format(len(stories[stories['type_id'] == 3])))

Number of stories referencing ID 3: 0.


We can safely remove it:

In [36]:
dataframes['story_type'] = df[df['id'] != 3]

### Language

Looking at the language file, all the rows are clean and it's safe to keep them as it is.

In [37]:
dataframes['language'].isnull().sum()

id      0
code    0
name    0
dtype: int64

### Brang group

In [38]:
dataframes['brand_group'].isnull().sum()

id                 0
name               0
year_began      2938
year_ended      3857
url             4701
publisher_id       0
notes_id        4615
dtype: int64

As we can see, the essential attributes don't have missing values.

In [39]:
dataframes['brand_group']['name'].value_counts().head()

Marvel           17
DC               10
Dargaud           8
A                 7
Panini Comics     6
Name: name, dtype: int64

However, we see that there are quite a lot of duplicates in the names. But if we look at the cell below, for the same names, we have each time different *publisher_id*s, so it makes sense to keep these duplicates.

In [40]:
dataframes['brand_group'][dataframes['brand_group']['name'] == 'Marvel']['publisher_id'].values

array([2105,  613, 3434,   78, 4720, 3174, 4437,  592, 3029, 8492, 7151,
       2195, 5905, 1798, 1977, 3655, 6917])

### Series Publication types
Obviously this table is ok.

In [41]:
dataframes['series_publication_type'].head()

,id,name
0,1,book
1,2,magazine
2,3,album


### Issue Reprint

We make sure there is no null rows in the reprint table:

In [42]:
dataframes['issue_reprint'].isnull().sum()

id                 0
origin_issue_id    0
target_issue_id    0
dtype: int64

### Indicia Publisher

For this table we need to make sure the *publisher_id* attribute is not null, which is the case:

In [43]:
dataframes['indicia_publisher'].isnull().sum()

id                 0
name               0
publisher_id       0
country_id         0
year_began      2612
year_ended      3563
is_surrogate       0
url             4711
notes_id        4282
dtype: int64

### Publisher

We need to make sure that every publisher as a name, which is the case

In [44]:
dataframes['publisher'].isnull().sum()

id               0
name             0
country_id       0
year_began    1275
year_ended    5560
url           7747
notes_id      6778
dtype: int64

### Stories

We did some cleaning at the beginning for cells that contained no information

### Issues

First we can drop the ISBN column as it doesn't bring more information than the Valid ISBN one:

In [53]:
dataframes['issue'].drop(['isbn'], axis=1, inplace=True)
dataframes['issue'].head()

,id,number,series_id,indicia_publisher_id,publication_date,price,page_count,indicia_frequency,valid_isbn,barcode,title,on_sale_date,rating,notes_id
0,1,[nn],1,NaN,1867,[none],96.0,NaN,NaN,NaN,NaN,1867,NaN,362469.0
1,2,[nn],2,NaN,1868,[none],134.0,NaN,NaN,NaN,NaN,NaN,NaN,362470.0
2,3,[nn],3,NaN,[circa 1870's],[none],16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,[nn],5,NaN,[circa 1870's],[none],14.0,NaN,NaN,NaN,NaN,1870,NaN,362471.0
4,5,[nn],6,NaN,[circa 1870's],none,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We see that the _number_ and _price_ columns don't have conventional NaN values, we fix that:

In [68]:
# Clean the number column
dataframes['issue']['number'] = dataframes['issue']['number'].replace('[nn]', np.nan).str.replace(r'\[nn\]', '')

# Clean the price column
dataframes['issue']['price'] = dataframes['issue']['price'].replace(['[None]', '[none]', 'none', '?'], np.nan)

dataframes['issue'].head()

,id,number,series_id,indicia_publisher_id,publication_date,price,page_count,indicia_frequency,valid_isbn,barcode,title,on_sale_date,rating,notes_id
0,1,NaN,1,NaN,1867,NaN,96.0,NaN,NaN,NaN,NaN,1867,NaN,362469.0
1,2,NaN,2,NaN,1868,NaN,134.0,NaN,NaN,NaN,NaN,NaN,NaN,362470.0
2,3,NaN,3,NaN,[circa 1870's],NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,5,NaN,[circa 1870's],NaN,14.0,NaN,NaN,NaN,NaN,1870,NaN,362471.0
4,5,NaN,6,NaN,[circa 1870's],NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Also, it seems we can remove '[' and ']' in *publication_date*:

In [83]:
dataframes['issue']['publication_date'] = dataframes['issue']['publication_date'].str.replace(r'\[|\]|\?', '')

Other columns seem ok.

### Series

In [84]:
dataframes['series']

,id,name,format,year_began,year_ended,publication_dates,publisher_id,country_id,language_id,color,dimensions,paper_stock,binding,publishing_format,publication_type_id,notes_id
0,1,Two Hundred Sketches Humorous and Grotesque,NaN,1867,1867.0,NaN,1,225,25,black and white cover and interior,"13-3/4"" tall x 11-7/16"" wide",NaN,hardcover,NaN,NaN,338930.0
1,2,A Bushel of Merry-Thoughts,NaN,1868,1868.0,NaN,2,75,25,color cover; black and white interior; Printed...,"6-1/2"" tall x 10"" wide",NaN,hardcover,NaN,NaN,NaN
2,3,Ye Veracious Chronicle of Gruff & Pompey in 7 ...,NaN,1870,1870.0,NaN,3,225,25,printed in color on one side blank on the othe...,"5-1/8"" tall x 3-3/8"" wide",folded cardboard booklet which unfolds into a ...,folded,one panel per page/fold,NaN,NaN
3,4,Further Adventures in Fools' Paradise with the...,NaN,1870,1870.0,NaN,4,225,25,brown cover with black illustrations; hand-col...,"10"" tall x 7-1/2"" wide",NaN,hardcover,NaN,NaN,338931.0
4,5,The Romance of a Hammock - As Recited by Mr. G...,NaN,1870,1870.0,NaN,5,225,25,printed in color on both sides read first one ...,"5-1/2"" tall x 3-1/2"" wide",folded light cardboard/heavy paper booklet whi...,NaN,giveaway,2.0,NaN
5,6,Miss Wiseman's Purchase / Mrs. Sillybilly's Pu...,NaN,1870,1870.0,NaN,6,225,25,color comics on one side; black and white text...,"5-7/8"" tall x 3-1/4"" wide",light cardboard or heavy paper?,NaN,NaN,NaN,338932.0
6,7,Max and Maurice -- A Juvenile History in Seven...,NaN,1871,1871.0,NaN,12,225,25,hand-coloured cover and interiors,"8-1/8"" tall x 5-5/8"" wide",NaN,hardcover,NaN,1.0,NaN
7,8,The Foreign Tour of Messrs. Brown Jones and Ro...,NaN,1860,1877.0,NaN,8,225,25,black and white interior printed one side of e...,"11-3/16"" tall x 9-1/2"" wide",with protective sheets over each page; gold gi...,hardcover,NaN,NaN,NaN
8,9,The Mischief Book,NaN,1880,1880.0,NaN,10,225,25,green and white cover; black and white interio...,"7-1/16"" tall x 10-3/4"" wide",NaN,hardcover,NaN,NaN,NaN
9,10,How Adolphus Slim-Jim Used Jackson's Best and ...,NaN,1880,1880.0,NaN,3,225,25,printed in color on one side blank on the othe...,"5-1/16"" tall x 3-3/8"" wide",folded cardboard booklet which unfolds into a ...,folded,NaN,1.0,NaN


We can capitalize some columns to increase a little bit consistency:

In [104]:
dataframes['series']['color'] = dataframes['series']['color'].str.capitalize()
dataframes['series']['dimensions'] = dataframes['series']['dimensions'].str.capitalize()
dataframes['series']['paper_stock'] = dataframes['series']['paper_stock'].str.capitalize()
dataframes['series']['binding'] = dataframes['series']['binding'].str.capitalize()
dataframes['series']['publishing_format'] = dataframes['series']['publishing_format'].str.capitalize()

dataframes['series'].head()

,id,name,format,year_began,year_ended,publication_dates,publisher_id,country_id,language_id,color,dimensions,paper_stock,binding,publishing_format,publication_type_id,notes_id
0,1,Two Hundred Sketches Humorous and Grotesque,NaN,1867,1867.0,NaN,1,225,25,Black and white cover and interior,"13-3/4"" tall x 11-7/16"" wide",NaN,Hardcover,NaN,NaN,338930.0
1,2,A Bushel of Merry-Thoughts,NaN,1868,1868.0,NaN,2,75,25,Color cover; black and white interior; printed...,"6-1/2"" tall x 10"" wide",NaN,Hardcover,NaN,NaN,NaN
2,3,Ye Veracious Chronicle of Gruff & Pompey in 7 ...,NaN,1870,1870.0,NaN,3,225,25,Printed in color on one side blank on the othe...,"5-1/8"" tall x 3-3/8"" wide",Folded cardboard booklet which unfolds into a ...,Folded,One panel per page/fold,NaN,NaN
3,4,Further Adventures in Fools' Paradise with the...,NaN,1870,1870.0,NaN,4,225,25,Brown cover with black illustrations; hand-col...,"10"" tall x 7-1/2"" wide",NaN,Hardcover,NaN,NaN,338931.0
4,5,The Romance of a Hammock - As Recited by Mr. G...,NaN,1870,1870.0,NaN,5,225,25,Printed in color on both sides read first one ...,"5-1/2"" tall x 3-1/2"" wide",Folded light cardboard/heavy paper booklet whi...,NaN,Giveaway,2.0,NaN


We can also translate the word _color_ again to increase consistency:

In [108]:
dataframes['series']['color'] = dataframes['series']['color'].replace(['Couleur', 'Colour', 'Farbig', 'Farger', 'Colori'], 'Color')

There are also some inconsistencies with the 'US' abbreviation in the dimensions:

In [112]:
dataframes['series']['dimensions'] = dataframes['series']['dimensions'].str.replace(r'us|u.s.|u. s.', 'US')

## Saving files

We can now save our clean and new tables, ready for database loading.

In [47]:
OUTPUT_PATH = os.path.join('..', 'data', 'clean')

#for name, df in dataframes.items():
#    df.to_csv(name.title() + '.csv', index=False, float_format='%.0f')


Also, we collect the max and average length of string attributes for each column of each table, in order to help use choosing right string lengths for the databas:

In [48]:
lengths = {}
for name, df in dataframes.items():
    tmp = {}
    for col in df.columns:
        col_type = df[col].dtype

        if (col_type == np.dtype('O') and type(df[col].dropna().iloc[0]) == str) or col_type == np.dtype(str):
            strs = df[col].dropna().str.len()
            tmp[col] = {'min': int(min(strs)),
                        'max': int(max(strs)),
                        'ave': int(sum(strs) / len(strs))}
    if len(tmp) > 0:        
        lengths[name] = tmp

In [49]:
with open(os.path.join(OUTPUT_PATH, 'lengths.json'), 'w') as file:
    json.dump([lengths], file)